In [119]:
## There is 5% Female and 18% Male in the artists -> data is shit!
## There is ~60% correct country formatting!

In [ ]:
import pandas as pd
import os

dataset_path = '/data/lfm2b-1mon'

users = pd.read_csv(os.path.join(dataset_path, 'users.tsv'), sep='\t')
users.rename(columns={'user_id': 'old_user_id'}, inplace=True)

test_dataset_path = dataset_path + '/listening_history_test.csv'
test_data = pd.read_csv(test_dataset_path)
test_data_user_augmented = test_data.merge(users, on='old_user_id')
test_data_user_augmented.head()
print(test_data_user_augmented.head())

tracks = pd.read_csv(os.path.join(dataset_path, 'tracks_augmented.csv'))
tracks.rename(columns={'track_id': 'old_item_id'}, inplace=True)
tracks.head()

test_data_user_item_augmented = test_data_user_augmented.merge(tracks, on='old_item_id')
test_data_user_item_augmented.head()

print('TRACKS\n\n\n', tracks)


test_data_user_item_augmented.rename(columns=
                                     {'country_x': 'user_country', 'age':'user_age', 'gender_x': 'user_gender', 'artist': 'artist_name', 'track':'track_name', 'gender_y':'artist_gender', 'country_y':'artist_country'}, inplace=True)
test_data_user_item_augmented.head()

train_dataset = pd.read_csv(os.path.join(dataset_path, 'listening_history_train.csv'))
train_dataset.head()

test_data_user_item_augmented['user_num_interactions'] = train_dataset.groupby('user_id').size()[test_data_user_item_augmented['user_id']].values
test_data_user_item_augmented['item_num_interactions'] = train_dataset.groupby('item_id').size()[test_data_user_item_augmented['item_id']].values

train_dataset_track_augmented = train_dataset.merge(tracks, on='old_item_id')

train_dataset_track_augmented['user_num_interactions'] = train_dataset.groupby('user_id').size()[train_dataset_track_augmented['user_id']].values
train_dataset_track_augmented['item_num_interactions'] = train_dataset.groupby('item_id').size()[train_dataset_track_augmented['item_id']].values

In [6]:
model_paths = []

In [ ]:
train_dataset_track_augmented['item_id'].max()

In [ ]:
import os
import torch
from rec_sys.tester import Tester
from experiment_helper import load_data
import argparse
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch import nn



models = {"b": "-", "kl": "-", "zs": "-"}

for i, k in enumerate(models.keys()):
    path_to_experiment = model_paths[i]
    config_path = os.path.join(path_to_experiment, 'params.json')
    with open(config_path, 'r') as f:
        config = json.load(f)
        
    config['device'] = 'cpu'
    config_dict = config.copy()
    config = argparse.Namespace(**config)
    data_loaders_dict = load_data(config, is_train=False)

    model_load_path = os.path.join(model_paths[i], 'checkpoint_000000/best_model.pth')
    tester = Tester(data_loaders_dict['test_loader'], config, model_load_path)
    models[k] = tester.model

In [ ]:
import pandas as pd
import os
import torch
from rec_sys.tester import Tester
from experiment_helper import load_data
import argparse
import json
import numpy as np
import pandas as pd
from torch import nn
from utilities.eval import Evaluator
import experiment_helper
from tqdm import tqdm


tester = Tester(data_loaders_dict['test_loader'], config, model_load_path)
test_loader = experiment_helper.load_data(config, is_train=False)['test_loader']


def get_outputs_for_eval(model):   
    outs_array_items = []
    outs_array_users = []

    for u_idxs, i_idxs, labels in test_loader:    
        test_loss = 0
        eval = Evaluator(u_idxs)
        out = model(u_idxs, i_idxs).detach()
        test_loss += model.loss_func(out, labels).item()
        out = nn.Sigmoid()(out)
        out = out.to('cpu')
        
        outs_array_items.append(np.array(torch.stack((i_idxs, labels, out), dim=2)))
        outs_array_users.append(np.array(u_idxs))
    return outs_array_items, outs_array_users



def get_processed_outputs(outs_array_items):
    pop_rank_tuples = []
    countries = {'US': [], 'GB': [], 'BR': [], 'NL': [], 'PL': [], 'AT': [], 'JP': [], 'CA': []}  # AT: austria , JP: japan, CA: canada, US: us, GB: gb, FR: france, RU: russia, IT: Italy
    genders = {'Male':[], 'Female':[]}
    for epoch_recom_list in tqdm(outs_array_items, desc="Processing epochs"):
        for i, users_recom_list in enumerate(tqdm(epoch_recom_list, desc="Processing users", leave=False)):
            sorted_indices = np.argsort(users_recom_list[:, 2])[::-1]
            ranked_outputs = np.empty_like(users_recom_list[:, 2])
            ranked_outputs[sorted_indices] = np.arange(0, len(users_recom_list[:, 2]))            
            users_recom_list = np.column_stack((users_recom_list, ranked_outputs.reshape(-1, 1)))
                        
            for item in users_recom_list:
                item_id = int(item[0])
                _ = item[1]
                item_output = item[2]
                item_rank = int(item[3])
                
                try:
                    item_pop = train_dataset_track_augmented[train_dataset_track_augmented['item_id'] == item_id]['item_num_interactions'].iloc[0]                    
                    pop_rank_tuples.append((item_id, item_pop, item_rank))
                except:
                    continue

                try:
                    item_country = train_dataset_track_augmented[train_dataset_track_augmented['item_id'] == item_id]['country'].iloc[0]
                    if item_country in list(countries.keys()):
                        countries[item_country].append(item_rank)
                except:
                    pass
                                    
                try:
                    item_gender = train_dataset_track_augmented[train_dataset_track_augmented['item_id'] == item_id]['gender'].iloc[0]
                    if item_gender in list(genders.keys()):
                        genders[item_gender].append(item_rank)
                except:
                    pass
        break
        
    return countries, genders, pop_rank_tuples


In [ ]:
model_outputs = {}

for i, model_path in enumerate(model_paths):
    print(i)
    path_to_experiment = model_path
    config_path = os.path.join(path_to_experiment, 'params.json')
    with open(config_path, 'r') as f:
        config = json.load(f)
    
    
    
    config['device'] = 'cpu'
    config_dict = config.copy()
    config = argparse.Namespace(**config)
    data_loaders_dict = load_data(config, is_train=False)

    model_load_path = os.path.join(model_path, 'checkpoint_000000/best_model.pth')
    tester = Tester(data_loaders_dict['test_loader'], config, model_load_path)
    model = tester.model

    test_loader = experiment_helper.load_data(config, is_train=False)['test_loader']
    print(f'model loaded / {i}')
    outs_model_items, outs_base_users = get_outputs_for_eval(model)
    print(f'got outputs / {i}')
    model_outputs_processed = get_processed_outputs(outs_model_items)
    model_outputs[i] = model_outputs_processed
    


In [ ]:
base_sorted_tuples = sorted(model_outputs[0][2], key=lambda x: x[0], reverse=False) # ascending popularity
cont_sorted_tuples = sorted(model_outputs[1][2], key=lambda x: x[0], reverse=False) # ascending popularity
zs_sorted_tuples = sorted(model_outputs[2][2], key=lambda x: x[0], reverse=False) # ascending popularity

percentiles = np.arange(0.1, 1.0, 0.1)
avg_ranks_b = []
avg_ranks_c = []
avg_ranks_z = []
for percentile in percentiles:
    below_p = int((percentile-.1) * len(cont_sorted_tuples))
    above_p = int(percentile * len(cont_sorted_tuples))
    print(below_p, above_p)
    
    average_rank_b = np.mean([t[1] for t in base_sorted_tuples[below_p:above_p]])
    average_rank_c = np.mean([t[1] for t in cont_sorted_tuples[below_p:above_p]])
    average_rank_z = np.mean([t[1] for t in zs_sorted_tuples[below_p:above_p]])

    avg_ranks_b.append(average_rank_b)
    avg_ranks_c.append(average_rank_c)
    avg_ranks_z.append(average_rank_z)
    
plt.xlabel('percentiles of popularity')    
plt.ylabel('avreage rank') 
plt.plot(percentiles, avg_ranks_b, label='baseline')
plt.plot(percentiles, avg_ranks_c, label='our contribution')
plt.plot(percentiles, avg_ranks_z, label='zs contribution')

plt.legend()

In [ ]:
model_output_ranks_per_country = {'baseline':model_outputs[0][0], 'kl': model_outputs[1][0], 'zs':model_outputs[2][0]}

averages = {}
for element, countries in model_output_ranks_per_country.items():
    averages[element] = {country: (sum(values) / len(values) if values else 0) for country, values in countries.items()}


# save averages dictionary to a file

import json

with open('averages_2.json', 'w') as f:
    json.dump(averages, f)
    

print(averages)
# Prepare data for plotting
countries = sorted({country for element in averages for country in averages[element]})
elements = sorted(averages.keys())

# Plotting
bar_width = 0.2
index = np.arange(len(countries))
fig, ax = plt.subplots()

elements[1], elements[2] = elements[2], elements[1]
for i, element in enumerate(elements):
    print(element)
    values = [averages[element][country] for country in countries]
    bar_positions = index + i * bar_width - 0.11
    color = 'blue' if i == 0 else 'green' if i == 1 else 'grey' # Color for US and GB
    ax.bar(bar_positions, values, bar_width, label=element, color=color)

# Add labels and title
ax.set_xlabel('Country')
ax.set_ylabel('Average Value')
ax.set_title('Average Values of Elements per Country')
ax.set_xticks(index + bar_width / len(elements))
ax.set_xticklabels(countries)
ax.legend()

# Show plot
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

# Load data
with open('averages_1.json', 'r') as f:
    averages = json.load(f)

# Groups
overrepresented = ['US', 'GB', 'CA']
underrepresented = ['BR', 'NL', 'PL']

# Labels and data preparation
labels = ['baseline', 'zs', 'kl']

# Data preparation for plotting
over_data = {label: [averages[label][country] for country in overrepresented] for label in labels}
under_data = {label: [averages[label][country] for country in underrepresented] for label in labels}

# Plot
fig, axs = plt.subplots(1, 2, figsize=(14, 6), sharey=True)

x = np.arange(len(overrepresented))
width = 0.2

# Overrepresented
for i, label in enumerate(labels):
    axs[0].bar(x + i * width, over_data[label], width, label=label)

axs[0].set_title('Overrepresented (US, GB, CA)')
axs[0].set_xticks(x + width)
axs[0].set_xticklabels(overrepresented)
axs[0].legend(title='Models')

# Underrepresented
x = np.arange(len(underrepresented))
for i, label in enumerate(labels):
    axs[1].bar(x + i * width, under_data[label], width, label=label)

axs[1].set_title('Underrepresented (BR, NL, PL)')
axs[1].set_xticks(x + width)
axs[1].set_xticklabels(underrepresented)
axs[1].legend(title='Models')

# General layout
plt.suptitle('Averages Comparison Across Different Groups and Methods')
plt.show()


In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

# Load data
with open('averages_1.json', 'r') as f:
    averages = json.load(f)

# Groups
overrepresented = ['US', 'GB', 'CA']
underrepresented = ['BR', 'NL', 'PL']

# Labels and data preparation
labels = ['baseline', 'zs', 'kl']
aaa = {'baseline': 'Baseline', 'zs': 'ZeroSum', 'kl': 'Our Model'}


# Data preparation for plotting
over_data = {label: [averages[label][country] for country in overrepresented] for label in labels}
under_data = {label: [averages[label][country] for country in underrepresented] for label in labels}




# Plot
fig, axs = plt.subplots(1, 2, figsize=(14, 6), sharey=True)

x = np.arange(len(overrepresented))
width = 0.2

# Overrepresented
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']  # Blue, Orange, Green
for i, label in enumerate(labels):
    axs[0].bar(x + i * width, over_data[label], width, label=aaa[label], color=colors[i])

axs[0].set_title('Overrepresented (The US, The UK, Canada)', fontsize=16)
axs[0].set_xticks(x + width * (len(labels) - 1) / 2)
axs[0].set_xticklabels(overrepresented, fontsize=18)
axs[0].set_ylabel('Average Rank of Items', fontsize=18)
axs[0].grid(True, linestyle='--', alpha=0.7)
axs[0].legend(title='Models', loc='lower right', fontsize=18)

# Underrepresented
x = np.arange(len(underrepresented))
for i, label in enumerate(labels):
    axs[1].bar(x + i * width, under_data[label], width, label=aaa[label], color=colors[i])

axs[1].set_title('Underrepresented (Bazil, Netherlands, Poland)', fontsize=16)
axs[1].set_xticks(x + width * (len(labels) - 1) / 2)
axs[1].set_xticklabels(underrepresented, fontsize=18)
axs[1].grid(True, linestyle='--', alpha=0.7)
axs[1].legend(title='Models', loc='lower right', fontsize=18)

# General layout
# plt.suptitle('Averages Comparison Across Different Groups and Methods', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


# Pre plotting, just metrics

In [ ]:
# save averages dictionary to a file

import json

with open('averages_0.json', 'w') as f:
    json.dump(averages, f)
    


In [21]:
def change_model(name):
    model = models[name]
    global user_embeddings, user_prototypes, item_embeddings, item_prototypes
    global user_prototype_model, item_prototype_model
    user_prototype_model = model.user_feature_extractor.model_1
    item_prototype_model = model.item_feature_extractor.model_1
    user_embeddings = user_prototype_model.embedding_ext.embedding_layer.weight[:]
    user_prototypes = user_prototype_model.prototypes
    item_embeddings = item_prototype_model.embedding_ext.embedding_layer.weight[:]
    item_prototypes = item_prototype_model.prototypes
    

In [22]:
import numpy as np
import torch.nn.functional as F

    
def distance_to_prototypes(indices, prototypes, embeddings, mode='average', k=5):

    def average_distance_to_prototypes(idx):
        cosine_similarity = F.cosine_similarity(embeddings[idx].unsqueeze(0), prototypes, dim=1)
        return 1 - cosine_similarity.mean().item()

    def distance_to_closest_prototype(idx):
        closest_prototype_index = F.cosine_similarity(embeddings[idx].unsqueeze(0), prototypes, dim=1).argmax().item()
        cosine_similarity = F.cosine_similarity(embeddings[idx].unsqueeze(0), prototypes[closest_prototype_index], dim=1)
        return 1 - cosine_similarity.mean().item() 
        
    def distance_to_top_k_closest_prototypes(idx):
        closest_prototype_indices = F.cosine_similarity(embeddings[idx].unsqueeze(0), prototypes, dim=1).topk(k).indices
        cosine_similarity = F.cosine_similarity(embeddings[idx].unsqueeze(0), prototypes[closest_prototype_indices], dim=1)
        return 1 - cosine_similarity.mean().item()
    
    if mode == 'average':
        distance_function = average_distance_to_prototypes
    elif mode == 'closest':
        distance_function = distance_to_closest_prototype
    elif mode == 'top_k':
        distance_function = distance_to_top_k_closest_prototypes

    average_distances = []
    for idx in indices:
        average_distances.append(distance_function(idx))
    return np.array(average_distances)

## average distance to prototypes accross genders

In [ ]:
male_user_ids = test_data_user_item_augmented.loc[test_data_user_item_augmented['user_gender']=='m']['user_id']
female_user_ids = test_data_user_item_augmented.loc[test_data_user_item_augmented['user_gender']=='f']['user_id']

In [ ]:
user_ks = np.arange(1, user_prototypes.shape[0], 5)

In [ ]:
male_distances = []
female_distances = []
for k in user_ks:
    male_distances.append(distance_to_prototypes(male_user_ids, user_prototypes, user_embeddings, mode='top_k', k=k).mean())
    female_distances.append(distance_to_prototypes(female_user_ids, user_prototypes, user_embeddings, mode='top_k', k=k).mean())

In [ ]:
plt.plot(user_ks, male_distances, 'b', label=f'Male Distances to k nearest prototypes, trained with k = {-1}')
plt.plot(user_ks, female_distances, 'r', label=f'Female Distances to k nearest prototypes, trained with k = {-1}')
plt.legend()

# locally save the image


In [ ]:
country_counts = test_data_user_item_augmented.groupby(['user_country']).size().sort_values(ascending=False)

pop_countries = list(country_counts[:3].index) 
mid_countries = list(country_counts[90:93].index)

pop_countries_ids = test_data_user_item_augmented[test_data_user_item_augmented['user_country'].isin(['IR'])]['user_id']
mid_countries_ids = test_data_user_item_augmented[test_data_user_item_augmented['user_country'].isin(mid_countries)]['user_id']

pop_distances = []
mid_distances = []
for k in user_ks:
    pop_distances.append(distance_to_prototypes(pop_countries_ids, user_prototypes, user_embeddings, mode='top_k', k=k).mean())
    mid_distances.append(distance_to_prototypes(mid_countries_ids, user_prototypes, user_embeddings, mode='top_k', k=k).mean())

In [ ]:
country_counts = test_data_user_item_augmented.groupby(['user_country']).size().sort_values(ascending=False)

pop_countries = list(country_counts[:3].index) 
mid_countries = list(country_counts[90:93].index)

pop_countries_ids = test_data_user_item_augmented[test_data_user_item_augmented['user_country'].isin(pop_countries)]['user_id']
mid_countries_ids = test_data_user_item_augmented[test_data_user_item_augmented['user_country'].isin(mid_countries)]['user_id']

pop_distances = []
mid_distances = []
for k in user_ks:
    pop_distances.append(distance_to_prototypes(pop_countries_ids, user_prototypes, user_embeddings, mode='top_k', k=k).mean())
    mid_distances.append(distance_to_prototypes(mid_countries_ids, user_prototypes, user_embeddings, mode='top_k', k=k).mean())

In [ ]:
import matplotlib.pyplot as plt
plt.plot(user_ks, pop_distances, 'b', label=f'Pop Distances to k nearest prototypes, trained with k = {-1}')
plt.plot(user_ks, mid_distances, 'r', label=f'Mid Distances to k nearest prototypes, trained with k = {-1}')
plt.legend()

In [ ]:
test_data_user_item_augmented['user_num_interactions'].describe(percentiles=[0.1, 0.25, 0.3, 0.4, 0.5, 0.6, 0.75, 0.8, 0.9, 0.99])

In [ ]:
user_ids_arr_popularity_groups = []

percentiles = {25: 57, 50: 130, 75: 245, 90: 395, 99: 749, 100: 1347}
per = [0] + list(percentiles.values())
percentilesss = list(percentiles.keys())
for i in range(len(per) - 1):
    user_ids_arr_popularity_groups.append(test_data_user_item_augmented.loc[(test_data_user_item_augmented['user_num_interactions'] > per[i]) & (test_data_user_item_augmented['user_num_interactions'] < per[i + 1])]['user_id'])

distances_arr = [[] for _ in range(len(user_ids_arr_popularity_groups))]
for user_group in range(len(user_ids_arr_popularity_groups)):
    for k in user_ks:
        distances_arr[user_group].append(distance_to_prototypes(user_ids_arr_popularity_groups[user_group], user_prototypes, user_embeddings, mode='top_k', k=k).mean())
distances_arr = np.array(distances_arr)

In [ ]:
plt.figure(figsize=(8, 6))
plt.rcParams['figure.dpi'] = 600


for i in range(len(user_ids_arr_popularity_groups)):
    plt.plot(user_ks, distances_arr[i]**5, label=f'Popularity Group Percentile - {percentilesss[i]}: {percentiles[percentilesss[i]]} Interactions, trained with k = {1}')
plt.legend()

In [ ]:
test_data_user_item_augmented['item_num_interactions'].describe(percentiles=[0.1, 0.25, 0.3, 0.4, 0.5, 0.6, 0.75, 0.8, 0.9, 0.99])

# --------------------------------------

# ITEM SIDE!

# --------------------------------------

In [23]:
change_model('b')

In [24]:
item_ks = np.arange(1, item_prototypes.shape[0], 5)


In [ ]:
test_data_user_item_augmented.head()

In [ ]:
test_data_user_item_augmented['artist_gender'].value_counts(), test_data_user_item_augmented['artist_gender'].nunique()

In [27]:
male_item_ids = test_data_user_item_augmented.loc[test_data_user_item_augmented['artist_gender']=='Male']['item_id']
female_item_ids = test_data_user_item_augmented.loc[test_data_user_item_augmented['artist_gender']=='Female']['item_id']

male_distances = []
female_distances = []
for k in item_ks:
    male_distances.append(distance_to_prototypes(male_item_ids, item_prototypes, item_embeddings, mode='top_k', k=k).mean())
    female_distances.append(distance_to_prototypes(female_item_ids, item_prototypes, item_embeddings, mode='top_k', k=k).mean())

In [ ]:
plt.plot(item_ks, male_distances, 'b', label=f'Male Distances to k nearest prototypes, trained with k = {1}')
plt.plot(item_ks, female_distances, 'r', label=f'Female Distances to k nearest prototypes, trained with k = {1}')
plt.legend()

In [ ]:
train_dataset_track_augmented['item_num_interactions'].describe(percentiles=[0.1, 0.25, 0.3, 0.4, 0.5, 0.6, 0.75, 0.8, 0.9, 0.99])

In [ ]:
test_data_user_item_augmented

In [ ]:
model_paths = []

models = {"b": "-"}

for i, k in enumerate(models.keys()):
    path_to_experiment = model_paths[i]
    config_path = os.path.join(path_to_experiment, 'params.json')
    with open(config_path, 'r') as f:
        config = json.load(f)
        
    config['device'] = 'cpu'
    config_dict = config.copy()
    config = argparse.Namespace(**config)
    data_loaders_dict = load_data(config, is_train=False)

    model_load_path = os.path.join(model_paths[i], 'checkpoint_000000/best_model.pth')
    tester = Tester(data_loaders_dict['test_loader'], config, model_load_path)
    models[k] = tester.model

change_model('b')


item_ks = np.arange(1, item_prototypes.shape[0], 5)
item_ids_arr_popularity_groups = []

percentiles = {0: 0, 25: 16, 95: 119, 100:689}
percentilesss = list(percentiles.keys())
per = list(percentiles.values())

for i in range(len(per) - 1):
    item_ids_arr_popularity_groups.append(test_data_user_item_augmented.loc[(test_data_user_item_augmented['item_num_interactions'] > per[i]) & (test_data_user_item_augmented['item_num_interactions'] <= per[i + 1])]['item_id'])
    
distances_arr = [[] for _ in range(len(item_ids_arr_popularity_groups))]
for item_group in range(len(item_ids_arr_popularity_groups)):
    for k in item_ks:
        distances_arr[item_group].append(distance_to_prototypes(item_ids_arr_popularity_groups[item_group], item_prototypes, item_embeddings, mode='top_k', k=k).mean())
distances_arr = np.array(distances_arr)

import plotly.graph_objects as go
import numpy as np

# Your data
ks = np.arange(1, item_prototypes.shape[0], 5)

# Create traces
fig = go.Figure()

fig.add_trace(go.Scatter(x=ks, y=distances_arr[0], mode='lines', name='first 25%'))
fig.add_trace(go.Scatter(x=ks, y=distances_arr[1], mode='lines', name='25% to 75%'))
fig.add_trace(go.Scatter(x=ks, y=distances_arr[2], mode='lines', name='top 5% popular'))

# Update layout
fig.update_layout(
    title='Average distance to prototypes',
    xaxis_title='number of prototypes',
    yaxis_title='average distance to prototypes',
    width=500,
    height=400,
)

fig.show()


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Your data
ks = np.arange(1, item_prototypes.shape[0], 5)

# Create traces
fig = go.Figure()

fig.add_trace(go.Scatter(x=ks, y=distances_arr[0], mode='lines', name='First 25%', line=dict(color='blue', width=6)))
fig.add_trace(go.Scatter(x=ks, y=distances_arr[1], mode='lines', name='25% to 75%', line=dict(color='green', width=6)))
fig.add_trace(go.Scatter(x=ks, y=distances_arr[2], mode='lines', name='Top 5% popular', line=dict(color='red', width=6)))

# Update layout
fig.update_layout(
    # title='Average Distance to Prototypes',
    xaxis_title='Number of Prototypes',
    yaxis_title='Average Distance to Prototypes',
    width=1000,
    height=600,
    font=dict(family='Arial', size=24),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    xaxis=dict(showline=True, showgrid=True, linecolor='rgb(0, 0, 0)', gridcolor='rgb(50,50,50)', linewidth=2, ticks='outside', tickfont=dict(family='Arial', size=18)),
    yaxis=dict(showline=True, showgrid=True, linecolor='rgb(0, 0, 0)', gridcolor='rgb(50,50,50)', linewidth=2, ticks='outside', tickfont=dict(family='Arial', size=18)),
    legend=dict(orientation='h', yanchor='top', y=1.1, xanchor='center', x=0.5),
)

fig.show()


In [ ]:
item_ks = np.arange(1, item_prototypes.shape[0], 5)
change_model('kl')
item_ids_arr_popularity_groups = []

percentiles = {0: 0, 25: 21, 95: 265, 100:689}
percentilesss = list(percentiles.keys())
per = list(percentiles.values())

for i in range(len(per) - 1):
    item_ids_arr_popularity_groups.append(test_data_user_item_augmented.loc[(test_data_user_item_augmented['item_num_interactions'] > per[i]) & (test_data_user_item_augmented['item_num_interactions'] <= per[i + 1])]['item_id'])
    
distances_arr = [[] for _ in range(len(item_ids_arr_popularity_groups))]
for item_group in range(len(item_ids_arr_popularity_groups)):
    for k in item_ks:
        distances_arr[item_group].append(distance_to_prototypes(item_ids_arr_popularity_groups[item_group], item_prototypes, item_embeddings, mode='top_k', k=k).mean())
distances_arr = np.array(distances_arr)

# each popular item has a prototype, and also in general they're closer to them.

# TODO:
- Performance parity accross sub groups
- Fairness in recommendation
    - Ginicoefficient
    - KL-divergence
    - Difference
    https://arxiv.org/pdf/2306.00403.pdf

    
- Discoverability Metrics


# Fairness Evaluation Set-up

In [ ]:
def swap_dic(metrics_dic):
    metrics_dic_new = {}
    for country, metrics in metrics_dic.items():
        for metric, value in metrics.items():
            # If the metric doesn't exist in the new dictionary, create it with an empty dictionary
            if metric not in metrics_dic_new:
                metrics_dic_new[metric] = {}
            # Add the value for the country under the corresponding metric
            metrics_dic_new[metric][country] = value
    return metrics_dic_new

In [9]:
from utilities.eval import Evaluator
import experiment_helper

def evaluate_for_user_subgroups(model, column='user_gender', subgroups=['m', 'f']):

    model.eval()

    test_loader = experiment_helper.load_data(config, is_train=False)['test_loader']
    groups = {subgroup:test_data_user_item_augmented[test_data_user_item_augmented[column] == subgroup]['user_id'] for subgroup in subgroups}


    # print(groups)
    group_losses = {}
    group_evals = {}

    for group_name, subgroup in groups.items():
        
        print('group name, subgroup', group_name, len(subgroup))
        
        test_loss = 0
        eval = Evaluator(0)
        
        # group name, subgroup US 687
        # (687,) torch.Size([256])
        for u_idxs, i_idxs, labels in test_loader:
            # in each batch, keep only the u_idxs that are in the subgroup
            mask = pd.Series(u_idxs).isin(subgroup)
            
            print(subgroup.shape, u_idxs.shape)
            break
            u_idxs, i_idxs, labels = u_idxs[mask], i_idxs[mask], labels[mask]
            if len(u_idxs) == 0:
                continue
            eval.n_users += len(u_idxs)
            out = model(u_idxs, i_idxs).detach()
            test_loss += model.loss_func(out, labels).item()
            out = nn.Sigmoid()(out)
            out = out.to('cpu')

            eval.eval_batch(out, group_name)

        group_losses[group_name] = test_loss
        group_evals[group_name] = eval

    return group_losses, group_evals

In [ ]:
test_data_user_item_augmented.head()

In [ ]:
from utilities.eval import Evaluator
import experiment_helper

def evaluate_for_item_subgroups(model, column='artist_gender', subgroups=['Male', 'Female']): # artist_country

    test_loader = experiment_helper.load_data(config, is_train=False)['test_loader']
    groups = {subgroup:test_data_user_item_augmented[test_data_user_item_augmented[column] == subgroup]['item_id'] for subgroup in subgroups}

    group_losses = {}
    group_evals = {}
                
    for u_idxs, i_idxs, labels in test_loader:    
        test_loss = 0
        eval = Evaluator(u_idxs)
        for group_name, subgroup in groups.items():                
            out = model(u_idxs, i_idxs).detach()
            test_loss += model.loss_func(out, labels).item()
            out = nn.Sigmoid()(out)
            out = out.to('cpu')

            i_idxs_flat = i_idxs.flatten()
            i_idxs_in_subgroup = pd.Series(i_idxs_flat).isin(subgroup).values.reshape(i_idxs.shape)
            # TODO MONDAY!!!!

            eval.eval_batch(out, group_name)

            group_losses[group_name] = test_loss
            group_evals[group_name] = eval
            
            if len(i_idxs_in_subgroup) == 0:
                print('no candidates')
                continue

            # calculate the rankings for the indices that are in i_idxs_in_subgroup            
    return group_losses, group_evals


group_losses, group_evals = evaluate_for_item_subgroups(model, column='artist_gender', subgroups=['Male', 'Female'])

In [ ]:
group_evals


In [ ]:
ge

In [ ]:
labels_flat[800:900]

In [ ]:
ar

In [ ]:
# what is the rank of the first item? in out_flat
ar = -out_flat[800:900]
ar.argsort()

In [ ]:
pd.Series(out_flat).isin(subg)

In [ ]:
subg

In [ ]:
f, m = evals['Female'], evals['Male']
f.get_results()

In [ ]:
m.get_results()

In [ ]:
evaluate_for_user_subgroups(model, column='user_gender', subgroups=['m', 'f'])

In [ ]:
import matplotlib.pyplot as plt

feature_column = 'user_country'
feature_subgroups = ['US', 'UK', 'IR', 'IT', 'CA', 'UA']
losses, metrics = evaluate_for_user_subgroups(model, column=feature_column, subgroups=feature_subgroups)
# metrics_dic = {}
# for subgroup in feature_subgroups:
#     metrics_dic[subgroup] = metrics[subgroup].get_results()

# metrics_dic = swap_dic(metrics_dic)

# countries = metrics_dic.keys()
# metrics = list(metrics_dic['hit_ratio@10'].keys())

# plt.figure(figsize=(8, 6))
# keys = list(metrics_dic.keys())
# metrics = list(metrics_dic[keys[0]].keys())

# bar_width = 0.07

# fig, ax = plt.subplots()
# for i in range(len(metrics)):
#     metric_values = [metrics_dic[key][metrics[i]] for key in keys]
#     ax.bar([x + i * bar_width for x in range(len(keys))], metric_values, bar_width, label=metrics[i])

# ax.set_xlabel('Metrics')
# ax.set_ylabel('Values')
# ax.set_title(f'Metrics for Different Subgroups, trained with k = {model_k}')
# ax.set_xticks([x + 0.2 for x in range(len(keys))])

# plt.xticks(fontsize=4)
# ax.set_xticklabels(keys)
# ax.legend()

# plt.tight_layout()
# plt.show()

In [ ]:
metrics_dic

In [ ]:
losses, metrics = evaluate_for_user_subgroups(model)
metrics_dic = {}
for subgroup in ['m', 'f']:
    metrics_dic[subgroup] = metrics[subgroup].get_results()
    
metrics_dic

In [ ]:
def plot_evaluate_user_subgroups(metrics_dic):
    
    metrics_dic = swap_dic(metrics_dic)
    plt.figure(figsize=(8, 6))
    keys = list(metrics_dic.keys())
    metrics = list(metrics_dic[keys[0]].keys())
    
    bar_width = 0.07
    num_metrics = len(metrics)
    index = range(len(keys))

    fig, ax = plt.subplots()
    for i in range(num_metrics):
        metric_values = [metrics_dic[key][metrics[i]] for key in keys]
        ax.bar([x + i * bar_width for x in index], metric_values, bar_width, label=metrics[i])

    ax.set_xlabel('Keys')
    ax.set_ylabel('Metrics')
    ax.set_title(f'Metrics for Different Keys, trained with k = {model_k}')
    plt.xticks(fontsize=4)
    ax.set_xticks([x + 0.1 for x in index])
    ax.set_xticklabels(keys)
    ax.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
plot_evaluate_user_subgroups(metrics_dic)

# Plotting

In [ ]:
user_prototype_model = model.user_feature_extractor.model_1
item_prototype_model = model.item_feature_extractor.model_1

In [ ]:
from utilities.explanations_utils import weight_visualization, get_top_k_items, tsne_plot

user_embeddings = user_prototype_model.embedding_ext.embedding_layer.weight[:]
user_prototypes = user_prototype_model.prototypes
tsne_plot(objects=user_embeddings.detach().numpy(), prototypes=user_prototypes.detach().numpy())

# Read Data

In [ ]:
test_data_user_item_augmented.head()

In [ ]:
from utilities.explanations_utils import weight_visualization, get_top_k_items, tsne_plot


user_indices = test_data_user_item_augmented['user_id'].values # sample the users you want to plot
# user_indices = user_indices[:100]

attributes_dict = {'user_gender':[], 'user_age':[], 'user_country':[],'user_num_interactions':[]}
test_data_user_item_augmented_arr = test_data_user_item_augmented[test_data_user_item_augmented['user_id'].isin(user_indices)][list(attributes_dict.keys())].values


In [ ]:
# indices
user_indices = user_indices.astype(int)
for i, key in enumerate(attributes_dict.keys()):
    print(i, key)
    attributes_dict[key] = test_data_user_item_augmented_arr[:, i]

In [ ]:
user_embeddings = user_prototype_model.embedding_ext.embedding_layer.weight[user_indices]
user_prototypes = user_prototype_model.prototypes

# User based
### 1. gender
### 2. age
### 3. creation year
### 4 num interactions
### 5. country

In [ ]:
tsne_plot(objects=user_embeddings.detach().numpy(), prototypes=user_prototypes.detach().numpy(), labels=attributes_dict['user_gender'], title='user_gender')

In [ ]:
tsne_plot(objects=user_embeddings.detach().numpy(), prototypes=user_prototypes.detach().numpy(), sizes=attributes_dict['user_age'].astype(float)**2, title='User Age')

In [ ]:
tsne_plot(objects=user_embeddings.detach().numpy(), prototypes=user_prototypes.detach().numpy(), sizes=(attributes_dict['creation_year'] - 2000).astype(float)**2, title='User Account Creation Year')

In [ ]:
tsne_plot(objects=user_embeddings.detach().numpy(), prototypes=user_prototypes.detach().numpy(), sizes=attributes_dict['user_num_interactions'].astype(float), title='User Interactions')

In [ ]:
country_counts = test_data_user_item_augmented.groupby(['user_country']).size().sort_values(ascending=False)
chosen_countries = list(country_counts[:3].index) + list(country_counts[40:43].index)

In [ ]:
chosen_countries

In [ ]:
# keep only the rows with the chosen countries
test_data_user_item_augmented_chosen_countries = test_data_user_item_augmented[test_data_user_item_augmented['user_country'].isin(chosen_countries)]
user_countries = test_data_user_item_augmented_chosen_countries['user_country'].values

user_indices = test_data_user_item_augmented_chosen_countries['user_id'].values
user_embeddings = user_prototype_model.embedding_ext.embedding_layer.weight[user_indices]

tsne_plot(objects=user_embeddings.detach().numpy(), prototypes=user_prototypes.detach().numpy(), labels=user_countries, title='User Countries')

# Tracks based
### 1. gender
### 2. num_interactions
### 3. country

In [ ]:
(test_data_user_item_augmented['artist_gender'].value_counts())

In [ ]:
from utilities.explanations_utils import weight_visualization, get_top_k_items, tsne_plot

item_indices = test_data_user_item_augmented['item_id'].values.astype(int) # sample the users you want to plot
# item_indices = item_indices[:100]

In [ ]:
attributes_dict = {'artist_gender':[], 'artist_country':[], 'item_num_interactions':[]}
test_data_user_item_augmented_arr = test_data_user_item_augmented[test_data_user_item_augmented['item_id'].isin(item_indices)][list(attributes_dict.keys())].values
for i, key in enumerate(attributes_dict.keys()):
    print(i, key)
    attributes_dict[key] = test_data_user_item_augmented_arr[:, i]

In [ ]:
item_embeddings = item_prototype_model.embedding_ext.embedding_layer.weight[item_indices]
item_prototypes = item_prototype_model.prototypes

In [ ]:
tsne_plot(item_embeddings.detach().numpy(), item_prototypes.detach().numpy())

In [ ]:
test_data_user_item_augmented[['item_id', 'item_num_interactions']]

In [ ]:
change_model('b')

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE


def tsne_plot(objects: np.ndarray, prototypes: np.ndarray, object_legend_text: str = 'Object', labels: np.ndarray = ['#74add1'], sizes: np.ndarray = [30], perplexity: int = 5,
              path_save_fig: str = None, title: str = 'TSNE plot for Object (user/item) Embeddings', us_indices=None, jp_indices=None):

    tsne = TSNE(perplexity=perplexity, metric='cosine', init='pca', learning_rate='auto', random_state=42)
    print('start')
    
    tsne_results = tsne.fit_transform(np.vstack([prototypes, objects]))
    
    print('finish')
    tsne_protos = tsne_results[:len(prototypes)]
    tsne_embeds = tsne_results[len(prototypes):]

    plt.figure(figsize=(10, 10), dpi=100)
    
    print('figs')
    # hsv
    cmap = plt.get_cmap('hsv')
    if len(labels) != 1:
        for i, label in enumerate(np.unique(labels)):
            plt.scatter(tsne_embeds[labels == label, 0], tsne_embeds[labels == label, 1], s=sizes, alpha=0.6, c=cmap(i / len(np.unique(labels))), label=label)
    else:
        plt.scatter(tsne_embeds[:, 0], tsne_embeds[:, 1], s=sizes, alpha=0.4, c=labels, label=object_legend_text, marker='o')
    plt.scatter(tsne_protos[:, 0], tsne_protos[:, 1], s=200, c='#d73027', alpha=0.5, label='Prototypes', marker='^')


    if us_indices is not None:
        chosen_points_US = tsne_embeds[us_indices]
        plt.scatter(chosen_points_US[:, 0], chosen_points_US[:, 1], s=220, c='blue', alpha=0.5, label='US Item', marker='x', linewidths=4)
    
    if jp_indices is not None:
        chosen_points_JP = tsne_embeds[jp_indices]
        plt.scatter(chosen_points_JP[:, 0], chosen_points_JP[:, 1], s=220, c='green', alpha=0.5, label='NL Item', marker='x', linewidths=4)


    print('done')
    plt.axis('off')
    plt.tight_layout()
    plt.legend(loc="upper left", prop={'size': 13})
    if path_save_fig:
        plt.savefig(path_save_fig, format='pdf')    


In [ ]:
train_dataset_track_augmented.head()

In [ ]:
df = train_dataset_track_augmented[['item_id', 'item_num_interactions', 'country']]
df.drop_duplicates(subset=['item_id'], inplace=True)
item_sizes = np.zeros(item_embeddings.shape[0])
len(item_sizes), item_embeddings.shape
for item_id in range(len(item_sizes)):
    try:
        item_sizes[item_id] = df[df['item_id'] == item_id]['item_num_interactions'].astype(float)
    except:
        item_sizes[item_id] = 0

In [ ]:
for item_id in range(len(item_sizes)):
    try:
        item_sizes[item_id] = df[df['item_id'] == item_id]['item_num_interactions'].astype(float)
    except:
        item_sizes[item_id] = 0

In [ ]:
# attributes_dict = {'country':[], 'item_num_interactions':[]}
# train_data = train_dataset_track_augmented[train_dataset_track_augmented['item_id'].isin(item_indices)][list(attributes_dict.keys())].values
# for i, key in enumerate(attributes_dict.keys()):
    # print(train_dataset_track_augmented[:, i])
    # attributes_dict[key] = train_dataset_track_augmented[:, i]
change_model('kl')
n = 4000

us_item_indices = train_dataset_track_augmented.loc[train_dataset_track_augmented.country == 'US']['item_id'].unique()
us_item_indices = us_item_indices[us_item_indices < n][15:20]

jp_item_indices = train_dataset_track_augmented.loc[train_dataset_track_augmented.country == 'NL']['item_id'].unique()
jp_item_indices = jp_item_indices[jp_item_indices < n][:5]

print('sss')
tsne_plot(item_embeddings.detach().numpy()[:n], item_prototypes.detach().numpy(), sizes=item_sizes[:n], title='Item Interactions', us_indices=us_item_indices, jp_indices=jp_item_indices)

In [ ]:
print('a')

In [ ]:
n = -len(item_embeddings)//10
n


In [ ]:
train_dataset_track_augmented.head()

In [ ]:
attributes_dict = {'country':[], 'item_num_interactions':[]}
train_data = train_dataset_track_augmented[train_dataset_track_augmented['item_id'].isin(item_indices)][list(attributes_dict.keys())].values
for i, key in enumerate(attributes_dict.keys()):
    print(train_dataset_track_augmented[:, i])
    attributes_dict[key] = train_dataset_track_augmented[:, i]

us_item_indices = train_dataset_track_augmented.loc[train_dataset_track_augmented.artist_country == 'US']['item_id'].values
jp_item_indices = train_dataset_track_augmented.loc[train_dataset_track_augmented.artist_country == 'JP']['item_id'].values


print('sss')

n = -5500
tsne_plot(item_embeddings[n:].detach().numpy(), item_prototypes.detach().numpy(), sizes=attributes_dict['item_num_interactions'].astype(float)[n:], title='Item Interactions', us_indices=us_item_indices, jp_indices=jp_item_indices)

In [ ]:
n = -len(item_embeddings)//10
tsne_plot(item_embeddings[-5000:].detach().numpy(), item_prototypes.detach().numpy(), sizes=attributes_dict['item_num_interactions'].astype(float)[-5000:], title='Item Interactions')#, us_indices=us_item_indices, jp_indices=jp_item_indices)

In [ ]:
tsne_plot(item_embeddings.detach().numpy(), item_prototypes.detach().numpy(), sizes=attributes_dict['item_num_interactions'].astype(float), title='Item Interactions', us_indices=us_item_indices, jp_indices=jp_item_indices)

In [ ]:
plt.show()

In [ ]:
# clean the data, only Female and Male genders
item_indices = test_data_user_item_augmented.loc[test_data_user_item_augmented.artist_gender.isin(['Male', 'Female'])]['item_id'].values
item_embeddings = item_prototype_model.embedding_ext.embedding_layer.weight[item_indices]
item_prototypes = item_prototype_model.prototypes

In [ ]:
attributes_dict['artist_gender'] = test_data_user_item_augmented[test_data_user_item_augmented['item_id'].isin(item_indices)]['artist_gender'].values

In [ ]:
attributes_dict

In [ ]:
tsne_plot(objects=item_embeddings.detach().numpy(), prototypes=item_prototypes.detach().numpy(), labels=attributes_dict['artist_gender'])

In [ ]:
country_counts = test_data_user_item_augmented.groupby(['user_country']).size().sort_values(ascending=False)
chosen_countries = list(country_counts[3:15].index)

In [ ]:
test_data_user_item_augmented.loc[test_data_user_item_augmented.artist_country.isin(chosen_countries)].groupby('artist_country').size()

In [ ]:
# clean the data, only Female and Male genders
item_indices = test_data_user_item_augmented.loc[test_data_user_item_augmented.artist_country.isin(chosen_countries)]['item_id'].values
item_embeddings = item_prototype_model.embedding_ext.embedding_layer.weight[item_indices]

item_prototypes = item_prototype_model.prototypes

attributes_dict['artist_country'] = test_data_user_item_augmented[test_data_user_item_augmented['item_id'].isin(item_indices)]['artist_country'].values

tsne_plot(objects=item_embeddings.detach().numpy(), prototypes=item_prototypes.detach().numpy(), labels=attributes_dict['artist_country'], title='TSNE FOR more COMMON COUNTRIES')

In [ ]:
import pandas as pd
import os
import torch
from rec_sys.tester import Tester
from experiment_helper import load_data
import argparse
import json
import numpy as np
import pandas as pd
from torch import nn
from utilities.eval import Evaluator
import experiment_helper



def calc_fairness_metrics(model_outputs):
    under_countries = ['BR', 'NL', 'PL']
    countries = [model_outputs[0][key] for key in under_countries]
    ranking_mean_under = np.mean([np.mean(country_avg_ranking) for country_avg_ranking in countries])
    
    over_countries = ['US', 'GB']
    countries = [model_outputs[0][key] for key in over_countries]
    ranking_mean_over = np.mean([np.mean(country_avg_ranking) for country_avg_ranking in countries])
    
    gender = [model_outputs[1][key] for key in ['Female']]
    ranking_mean_female = np.mean([np.mean(country_avg_ranking) for country_avg_ranking in gender])

    gender = [model_outputs[1][key] for key in ['Male']]
    ranking_mean_male = np.mean([np.mean(country_avg_ranking) for country_avg_ranking in gender])

    rank_tuples = model_outputs[2]
    lt_normal_perc = train_dataset_track_augmented['item_num_interactions'].quantile(0.2)
    lt_log_perc = pd.Series(np.log(train_dataset_track_augmented['item_num_interactions'])).quantile(0.2)**2

    lt_array = np.array([(x, y < lt_normal_perc, y < lt_log_perc)  for  (id, x, y) in rank_tuples])
    ranking_mean_for_normal_lt = np.mean(lt_array[lt_array[:, 1], 0])
    ranking_mean_for_log_lt = np.mean(lt_array[lt_array[:, 2], 0])


    lt_array = np.array([(id, y < lt_normal_perc, y < lt_log_perc)  for  (id, x, y) in rank_tuples])
    unique_values_normal = len(np.unique(lt_array[lt_array[:, 1], 0]))
    unique_values_log = len(np.unique(lt_array[lt_array[:, 2], 0]))
    
        
    fairness_metrics = {'under_countries_avg_rank': ranking_mean_under, 'over_countries_avg_rank': ranking_mean_over, 'mean_rank_female': ranking_mean_female, 'mean_rank_male': ranking_mean_male, 'mean_rank_lt_normal': ranking_mean_for_normal_lt, 'mean_rank_lt_log':ranking_mean_for_log_lt, 'unique_values_normal': unique_values_normal, 'unique_values_log': unique_values_log}
    return fairness_metrics

def get_outputs_for_eval(model):   
    outs_array_items = []
    outs_array_users = []

    for u_idxs, i_idxs, labels in test_loader:    
        test_loss = 0
        eval = Evaluator(u_idxs)
        out = model(u_idxs, i_idxs).detach()
        test_loss += model.loss_func(out, labels).item()
        out = nn.Sigmoid()(out)
        out = out.to('cpu')
        
        outs_array_items.append(np.array(torch.stack((i_idxs, labels, out), dim=2)))
        outs_array_users.append(np.array(u_idxs))
    return outs_array_items, outs_array_users



def get_processed_outputs(outs_array_items):
    pop_rank_tuples = []
    countries = {'US': [], 'GB': [], 'BR': [], 'NL': [], 'PL': [], 'CA': [], 'IR': []}  # AT: austria , JP: japan, CA: canada, US: us, GB: gb, FR: france, RU: russia, IT: Italy
    genders = {'Male':[], 'Female':[]}
    for epoch_recom_list in outs_array_items:
        for users_recom_list in epoch_recom_list:
            sorted_indices = np.argsort(users_recom_list[:, 2])[::-1]
            ranked_outputs = np.empty_like(users_recom_list[:, 2])
            ranked_outputs[sorted_indices] = np.arange(0, len(users_recom_list[:, 2]))            
            users_recom_list = np.column_stack((users_recom_list, ranked_outputs.reshape(-1, 1)))
                        
            for item in users_recom_list:
                item_id = int(item[0])
                _ = item[1]
                item_output = item[2]
                item_rank = int(item[3])
                
                
                try:
                    item_pop = train_dataset_track_augmented[train_dataset_track_augmented['item_id'] == item_id]['item_num_interactions'].iloc[0]                    
                    pop_rank_tuples.append((item_id, item_pop, item_rank))
                except:
                    continue

                try:
                    item_country = train_dataset_track_augmented[train_dataset_track_augmented['item_id'] == item_id]['country'].iloc[0]
                    if item_country in list(countries.keys()):
                        countries[item_country].append(item_rank)
                except:
                    pass
                                    
                try:
                    item_gender = train_dataset_track_augmented[train_dataset_track_augmented['item_id'] == item_id]['gender'].iloc[0]
                    if item_gender in list(genders.keys()):
                        genders[item_gender].append(item_rank)
                except:
                    pass
        break
    return countries, genders, pop_rank_tuples


dataset_path = 'data/lfm2b-1mon'
users = pd.read_csv(os.path.join(dataset_path, 'users.tsv'), sep='\t')
users.rename(columns={'user_id': 'old_user_id'}, inplace=True)
test_dataset_path = dataset_path + '/listening_history_test.csv'
test_data = pd.read_csv(test_dataset_path)
test_data_user_augmented = test_data.merge(users, on='old_user_id')
tracks = pd.read_csv(os.path.join(dataset_path, 'tracks_augmented.csv'))
tracks.rename(columns={'track_id': 'old_item_id'}, inplace=True)
test_data_user_item_augmented = test_data_user_augmented.merge(tracks, on='old_item_id')
test_data_user_item_augmented.rename(columns={'country_x': 'user_country', 'age':'user_age', 'gender_x': 'user_gender', 'artist': 'artist_name', 'track':'track_name', 'gender_y':'artist_gender', 'country_y':'artist_country'}, inplace=True)
train_dataset = pd.read_csv(os.path.join(dataset_path, 'listening_history_train.csv'))
test_data_user_item_augmented['user_num_interactions'] = train_dataset.groupby('user_id').size()[test_data_user_item_augmented['user_id']].values
test_data_user_item_augmented['item_num_interactions'] = train_dataset.groupby('item_id').size()[test_data_user_item_augmented['item_id']].values
train_dataset_track_augmented = train_dataset.merge(tracks, on='old_item_id')
train_dataset_track_augmented['user_num_interactions'] = train_dataset.groupby('user_id').size()[train_dataset_track_augmented['user_id']].values
train_dataset_track_augmented['item_num_interactions'] = train_dataset.groupby('item_id').size()[train_dataset_track_augmented['item_id']].values
print(train_dataset_track_augmented.head())

# TODO ADD args

model_path = """ray_results/user_item_proto_lfm2b-1mon_cn_38210573_2024-4-5_15-12-24.486415/user_item_proto_lfm2b-1mon_cn_38210573_36e2aa7f_6_batch_size=313,data_path"""
path_to_experiment = model_path
config_path = os.path.join(path_to_experiment, 'params.json')
with open(config_path, 'r') as f:
    config = json.load(f)
    
config['device'] = 'cpu'
config_dict = config.copy()
config = argparse.Namespace(**config)
data_loaders_dict = load_data(config, is_train=False)

model_load_path = os.path.join(model_path, 'checkpoint_000000/best_model.pth')
tester = Tester(data_loaders_dict['test_loader'], config, model_load_path)
model = tester.model

test_loader = experiment_helper.load_data(config, is_train=False)['test_loader']

outs_model_items, outs_base_users = get_outputs_for_eval(model)
model_outputs_processed = get_processed_outputs(outs_model_items)

metrics = calc_fairness_metrics(model_outputs_processed)


print(metrics)
# TODO: write to file
file_name = 'jobid'
LOGS_PATH = f"logs/FAIRNESS_METRICS/{0}"
with open(LOGS_PATH.format(file_name), "w") as json_file:
    json.dump(metrics, json_file)



In [ ]:
metrics = calc_fairness_metrics(model_outputs_processed)
metrics